# Introduction

In this notebook, we implement a simpler version of the BUC (Bottom up cubing) algorithm. 

The original BUC algorithm is a highly optimized and practically efficient algorithm to materialize the entire data cube given a fact table. We instead experiment with a simpler version, called `BUC_rec` (BUC recursive), based on our teaching slides. The implementation here emphasize more on the conceptual simplicity rather than efficiency. 

Once you understand `BUC_rec`, you may try to understand `BUC` if you want to learn more by yourself. 

The recursive formula used in `BUC_rec` is: 

$$
Cube(R, \{A,B,C,\ldots,M\}) = \bigcup_{i \in \{1, 2, \ldots, m, \text{ALL}\}} Cube(R_i, \{B,C,\ldots,M\}) 
$$
where $R_i$ is $\pi_{B,C,\ldots,M}(\sigma_{A = a_i}(R))$ and $R_{\text{ALL}}$ is $\pi_{B,C,\ldots,M}(R)$.

In [1]:
import sys
import pandas as pd
import numpy as np


ALL = -1

# DEBUG = True
DEBUG = False

##============================================================

# Data file format: 
# * tab-delimited input file
# * 1st line: dimension names and the last dimension is assumed to be the measure
# * rest of the lines: data values. 
def read_data(filename):
    df = pd.read_csv(filename, sep='\t')
    dims = df.shape[1] - 1 # the last dim is the measure
    return (df, dims)

def dump_input2(input):
    if DEBUG: 
        print("\n.. BUC_rec invoked on:")
        print(input)
        print("......................\n")
        
# helper functions
def project_data(input, d):
    # Return only the d-th column of INPUT
    return input.iloc[:, d]

def select_data(input, d, val):
    # SELECT * FROM INPUT WHERE INPUT.d = VAL
    col_name = input.columns[d]
    return input[input[col_name] == val]

def remove_first_dim(input):
    # Remove the first dim of the input
    return input.iloc[:, 1:]

def slice_data_dim0(input, v):
    # syntactic sugar to get R_{ALL} in a less verbose way
    df_temp = select_data(input, 0, v)
    return remove_first_dim(df_temp)

def output(val):
    print('=>\t{}'.format(val))


In [3]:
# psa test
data, d = read_data('./asset/c_.txt')
print(d)
data

3


,A,B,C,M
0,Sydney,2005,PS2,1400
1,Sydney,2006,PS2,1500
2,Sydney,2006,Wii,500
3,Melbourne,2005,XBOX 360,1700


In [4]:
data, d = read_data('./asset/a_.txt')
print(d)
data

2


,A,B,M
0,1,1,20
1,2,1,50
2,1,2,30
3,1,3,40


In [60]:
data.shape # (4, 3)
# data.iloc[:] 
# data.iloc[:, 1:]

(4, 3)

In [80]:
# print(project_data(data, 0))
print(project_data(data, 0))
# print(sum(project_data(data, 0)))

0    1
1    2
2    1
3    1
Name: A, dtype: int64


In [66]:
select_data(data, 1, 3)

,A,B,M
3,1,3,40


In [25]:
# slice_data_dim0(data, 2)
slice_data_dim0(data, 1)

,B,M
0,1,20
2,2,30
3,3,40


Now we can implement the `buc_rec()` algorithm and test it.

In [7]:
def buc_helper(input, output_df=[], index=''):

    dump_input2(input)
    dims = input.shape[1]
    
    if dims == 1:
        # only the measure dim
        input_sum = sum( project_data(input, 0) )
        output_df.append((index, input_sum))
#         output(input_sum)
    else:
        # the general case
        dim0_vals = set(project_data(input, 0).values)
        for dim0_v in dim0_vals:
            if index != '':
                new_index = ',' + str(dim0_v)
            else:
                new_index = str(dim0_v)
            sub_data = slice_data_dim0(input, dim0_v)
            buc_helper(sub_data, output_df, index + new_index)
            
        ## for R_{ALL}
        sub_data = remove_first_dim(input)
        buc_helper(sub_data, output_df, index + ',ALL')
    return output_df

def buc_rec(input):
    temp_data_raw = buc_helper(input)
    pd_raw_data = []
    for i, j in temp_data_raw:
        aList = list(filter(lambda x: x != '', i.split(',')))
        aList.append(j)
        pd_raw_data.append(aList)

    return pd.DataFrame.from_records(pd_raw_data, columns=['A','B','C','M'])

data

,A,B,C,M
0,Sydney,2005,PS2,1400
1,Sydney,2006,PS2,1500
2,Sydney,2006,Wii,500
3,Melbourne,2005,XBOX 360,1700


In [10]:
# data2, d = read_data('./asset/b_.txt')
# print(data2)
export_df = buc_rec(data)
print(export_df)
# buc_rec(data.head(1))
# input = data.head(1)
# input = data
# dim0_vals = set(project_data(input, 0).values)
# print(dim0_vals)
# print(slice_data_dim0(input, list(dim0_vals)[0]))
# print('\n')
# print(slice_data_dim0(input, list(dim0_vals)[1]))
# print('\n')
# print(remove_first_dim(input))
# print(buc_rec(remove_first_dim(input)))
# export_df.to_csv('c_buc.csv', sep=',', encoding='utf-8')

            A     B         C     M
0   Melbourne  2005  XBOX 360  1700
1   Melbourne  2005       ALL  1700
2   Melbourne   ALL  XBOX 360  1700
3   Melbourne   ALL       ALL  1700
4      Sydney  2005       PS2  1400
5      Sydney  2005       ALL  1400
6      Sydney  2006       PS2  1500
7      Sydney  2006       Wii   500
8      Sydney  2006       ALL  2000
9      Sydney   ALL       PS2  2900
10     Sydney   ALL       Wii   500
11     Sydney   ALL       ALL  3400
12        ALL  2005       PS2  1400
13        ALL  2005  XBOX 360  1700
14        ALL  2005       ALL  3100
15        ALL  2006       PS2  1500
16        ALL  2006       Wii   500
17        ALL  2006       ALL  2000
18        ALL   ALL       PS2  2900
19        ALL   ALL  XBOX 360  1700
20        ALL   ALL       Wii   500
21        ALL   ALL       ALL  5100
22  Melbourne  2005  XBOX 360  1700
23  Melbourne  2005       ALL  1700
24  Melbourne   ALL  XBOX 360  1700
25  Melbourne   ALL       ALL  1700
26     Sydney  2005       PS

With the following pivot table, we can easily see the output is correct (i.e., all the (non-empty) aggregates are computed). 

But did you notice anything else interesting? 

In [20]:
data.pivot_table(index = ['A'], columns = ['B'], aggfunc = np.sum, margins = True)

M                   
B       1     2     3    All
A                           
1    20.0  30.0  40.0   90.0
2    50.0   NaN   NaN   50.0
All  70.0  30.0  40.0  140.0

## Exercise

The above implementation is not complete in that the aggregated values are not associated with their "coordinates". We would like to have sth like 

<pre>
1      	1      	=>     	20
1      	2      	=>     	30
1      	3      	=>     	40
1      	*      	=>     	90
2      	1      	=>     	50
2      	*      	=>     	50
*      	1      	=>     	70
*      	2      	=>     	30
*      	3      	=>     	40
*      	*      	=>     	140
</pre>

Your task is to enhance the implementation of `buc_rec` so that you can generate the above more readable output. 

In [137]:
a = [('1,1', 20),
 ('1,2', 30),
 ('1,3', 40),
 ('1,ALL', 90),
 ('2,1', 50),
 ('2,ALL', 50),
 (',ALL,1', 70),
 (',ALL,2', 30),
 (',ALL,3', 40),
 (',ALL,ALL', 140)]

pd_raw_data = []
for i, j in a:
    aList = list(filter(lambda x: x != '', i.split(',')))
    aList.append(j)
    pd_raw_data.append(aList)
    
pd.DataFrame.from_records(pd_raw_data, columns=['A','B','M'])
    
#     pd.DataFrame.from_records(
#         remove_list_items(rows), 
#         columns=["State", "R", "D", "incumbent"])

,A,B,M
0,1,1,20
1,1,2,30
2,1,3,40
3,1,ALL,90
4,2,1,50
5,2,ALL,50
6,ALL,1,70
7,ALL,2,30
8,ALL,3,40
9,ALL,ALL,140


In [146]:
[1, 2].index(2)

1